# **Heat Exchanger Pipe Length Calculator**

This Jupyter Notebook calculates the required pipe length for a ground-coupled heat exchanger based on user-defined parameters.
You can specify the inlet air temperature, ground temperature, air velocity, pipe diameter, and the target temperature difference from the ground temperature.

---
### **Key Features**
- **Dynamic Inputs**: Adjust parameters interactively to see how they affect the required pipe length.
- **Mass Flow in m³/h**: Displays the air mass flow rate in cubic meters per hour for better visualization.
- **Handles Both Heating and Cooling**: Ensures proper calculations for both positive and negative temperature differences.

---
### **How to Use**
1. Adjust the sliders for input values.
2. Run the notebook cell to see updated results.
3. View key outputs, including required pipe length and heat transfer properties.


In [ ]:
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider

def calculate_pipe_length_dynamic(T_in, T_ground, v, D, delta_T_target):
    # Constants
    rho_air = 1.2  # Air density (kg/m³)
    cp_air = 1005  # Specific heat capacity of air (J/(kg·K))
    k_air = 0.025  # Thermal conductivity of air (W/(m·K))
    mu_air = 1.81e-5  # Dynamic viscosity of air (kg/(m·s))
    Pr_air = 0.7  # Prandtl number (dimensionless)

    # Target outlet temperature based on the target temperature difference
    if T_in > T_ground:  # Cooling case
        T_out = T_ground + delta_T_target
    else:  # Heating case
        T_out = T_ground - delta_T_target

    # Effectiveness
    effectiveness = (T_out - T_in) / (T_ground - T_in)

    # Cross-sectional area and mass flow rate
    A_cross = np.pi * (D / 2)**2  # Cross-sectional area (m²)
    m_dot = rho_air * v * A_cross  # Mass flow rate (kg/s)
    volumetric_flow_rate_m3h = m_dot / rho_air * 3600  # Volumetric flow rate (m³/h)

    # Reynolds number
    Re = (rho_air * v * D) / mu_air
    if Re > 2300:
        flow_type = "Turbulent"
        Nu = 0.023 * (Re**0.8) * (Pr_air**0.3)  # Turbulent Nusselt number
    else:
        flow_type = "Laminar"
        Nu = 3.66  # Laminar Nusselt number

    # Convective heat transfer coefficient
    h_conv = (Nu * k_air) / D

    # Overall heat transfer coefficient (soil properties assumed)
    t_tube = 0.003  # Tube wall thickness (m)
    k_tube = 0.4  # Thermal conductivity of tube material (W/(m·K))
    k_soil = 1.5  # Soil thermal conductivity (W/(m·K))
    R_cond = t_tube / (k_tube * np.pi * D)
    R_soil = 1 / (2 * np.pi * k_soil)
    U = 1 / (1 / h_conv + R_cond + R_soil)

    # Pipe length calculation
    L = - (m_dot * cp_air) / (U * np.pi * D) * np.log(1 - effectiveness)

    # Results
    return {
        "Flow Type": flow_type,
        "Reynolds Number": Re,
        "Convective Heat Transfer Coefficient (h_conv)": h_conv,
        "Overall Heat Transfer Coefficient (U)": U,
        "Mass Flow Rate (m_dot, kg/s)": m_dot,
        "Volumetric Flow Rate (m³/h)": volumetric_flow_rate_m3h,
        "Target Outlet Temperature (T_out)": T_out,
        "Effectiveness": effectiveness,
        "Required Pipe Length (L)": L
    }

def display_results(T_in, T_ground, v, D, delta_T_target):
    results = calculate_pipe_length_dynamic(T_in, T_ground, v, D, delta_T_target)
    print("Heat Exchanger Pipe Length Calculation")
    print("-" * 40)
    for key, value in results.items():
        if isinstance(value, float):
            print(f"{key}: {value:.2f}")
        else:
            print(f"{key}: {value}")

# Interactive sliders for input parameters
interact(
    display_results,
    T_in=FloatSlider(value=20, min=-30, max=50, step=1, description="T_in (°C)"),
    T_ground=FloatSlider(value=10, min=-10, max=40, step=1, description="T_ground (°C)"),
    v=FloatSlider(value=1, min=0.1, max=5, step=0.1, description="Velocity (m/s)"),
    D=FloatSlider(value=0.11, min=0.05, max=0.5, step=0.01, description="Diameter (m)"),
    delta_T_target=FloatSlider(value=2, min=0.1, max=10, step=0.1, description="Delta T (°C)")
);